In [10]:
import os
from Corpus import Corpus

theme = "climatechange"
filename = f"{theme}_corpus_new.csv"

corpus = Corpus(f"Corpus_{theme}")

# --- Si le fichier existe : charger ---
if os.path.exists(filename):
    print(f"\nChargement du corpus depuis {filename}...")
    corpus.load(filename)

# --- Sinon : construire le corpus ---
else:
    print("\nAucun fichier trouvé. Construction du corpus...")

    #  extraction Reddit + Arxiv
    # (les blocs 2 et 3)



Aucun fichier trouvé. Construction du corpus...


In [11]:
import praw
from datetime import datetime

reddit = praw.Reddit(
    client_id='_qZmFflED2oGl3LmqZvqBg',
    client_secret='1wQi5AamyCOBlq5HQqbpu85PDRjySQ',
    user_agent='projet_api'
)

mot_cles = [
    'climate', 'climatechange', 'environment', 'sustainability',
    'carbonfootprint', 'ecology', 'greenenergy', 'climateaction'
]
print("\n--- Extraction Reddit ---")

for mot in mot_cles:
    posts = reddit.subreddit(mot).top(limit=20)
    for post in posts:
        text = post.selftext.replace("\n", " ").strip()
        if text == "":
            continue

        title = post.title
        author_name = str(post.author) if post.author else "Unknown"
        date = datetime.fromtimestamp(post.created_utc).strftime("%Y-%m-%d")
        url = f"https://reddit.com{post.permalink}"
        texte = text
        corpus.add_document(
            post.title,
            str(post.author),
            datetime.fromtimestamp(post.created_utc).strftime("%Y-%m-%d"),
            f"https://reddit.com{post.permalink}",
            texte
        )



--- Extraction Reddit ---


In [ ]:
import urllib.request, xmltodict, ssl, certifi

ssl_context = ssl.create_default_context(cafile=certifi.where())
url = f"https://export.arxiv.org/api/query?search_query=all:{theme}&start=0&max_results=10"

print("\n--- Extraction Arxiv ---")

data = xmltodict.parse(urllib.request.urlopen(url, context=ssl_context).read())
entries = data["feed"]["entry"]
if isinstance(entries, dict):
    entries = [entries]

for entry in entries:
    auteurs = entry["author"]
    if isinstance(auteurs, dict):
        auteurs = [auteurs]
    auteur_str = ", ".join(a["name"] for a in auteurs)

    corpus.add_document(
        entry["title"].strip(),
        auteur_str,
        entry["published"],
        entry["id"],
        entry["summary"].strip()
    )


In [12]:
if not os.path.exists(filename):
    print("\nSauvegarde du corpus...")
    corpus.save(filename)



Sauvegarde du corpus...
Corpus sauvegardé dans climatechange_corpus_new.csv
